# answer detection question

In [3]:
import requests
from PIL import Image
import torch
from transformers import AutoProcessor, LlavaNextForConditionalGeneration

# Load the model in half-precision
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, device_map="auto")
processor = AutoProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")



Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.04s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [5]:
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "I will give you some pictures with red boxes in them. Please answer the questions according to the contents in the boxes"},
        ],
    },
    {
        "role": "assistant",
        "content": [{"type": "text", "text": "Got it! You can send me the picture and I will answer the question based on the content in the red box."},]
    },
    {

        "role": "user",
        "content": [
            {'type': 'image'},
            {"type": "text", "text": "What's the color of the car"},
        ],
    },
]

image_input = Image.open('/root/videollm-online/data/preprocess/output.png')

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
prompts = [prompt]

inputs = processor(images=[image_input], text=prompts, padding=True, return_tensors="pt").to(model.device)
generate_ids = model.generate(**inputs, max_new_tokens=1200)
output = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [6]:
print(output[0].split('[/INST]')[-1])

 The color of the car in the image is white. 


# caption video

In [1]:
import av
import torch
import numpy as np
from transformers import LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor

# Load the model in half-precision
model = LlavaNextVideoForConditionalGeneration.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf", torch_dtype=torch.float16, device_map="auto")
processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")



/root/miniconda3/envs/videollm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 3/3 [00:48<00:00, 16.25s/it]


(8,)
(8, 1080, 1440, 3)


/root/miniconda3/envs/videollm/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789115765/work/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(value)
/root/miniconda3/envs/videollm/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be r

['USER: \nplease describe the video. ASSISTANT: The video shows a view from the back seat of a car, likely a taxi or rideshare vehicle, as indicated by the presence of a driver in the rearview mirror. The car is moving along a street with palm trees on both sides, suggesting a warm climate. The driver']


In [22]:
from decord import VideoReader
from tqdm import tqdm

# Load the video as an np.array, sampling uniformly 8 frames (can sample more for longer videos)
video_path = '/root/videollm-online/datasets/ego4d/v2/full_scale_1fps/9ff8c35c-bd28-436f-b35b-ee460f983a67.mp4'
vr = VideoReader(video_path)

video_descriptions = []
interval = 32
each_turn_step = 4
for i in tqdm(range(0, vr._num_frame, interval)):
    video = vr.get_batch(range(i, min(vr._num_frame - 1, i+interval), each_turn_step))
    
    print(video.shape)
    conversation = [
        {

            "role": "user",
            "content": [
                {"type": "text", "text": "please describe the video."},
                {"type": "video"},
                ],
        },
    ]

    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor(text=prompt, videos=video, return_tensors="pt")

    out = model.generate(**inputs, max_new_tokens=1200)
    output = processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    assitant_output = output[0].split('ASSISTANT: ')[-1]
    video_descriptions.append(assitant_output)

  0%|          | 0/21 [00:00<?, ?it/s]

torch.Size([8, 1080, 1440, 3])


  5%|▍         | 1/21 [00:49<16:21, 49.10s/it]

torch.Size([8, 1080, 1440, 3])


 10%|▉         | 2/21 [01:40<15:53, 50.20s/it]

...

 90%|█████████ | 19/21 [16:43<01:25, 42.94s/it]

torch.Size([8, 1080, 1440, 3])


 95%|█████████▌| 20/21 [17:21<00:41, 41.45s/it]

torch.Size([6, 1080, 1440, 3])


100%|██████████| 21/21 [18:00<00:00, 51.43s/it]


In [26]:
example = ''
for i, desc in enumerate(video_descriptions):
    example += f"clip {i}: {desc}\n"
print(example)

clip 0: The video shows the interior of a vehicle, specifically the backseat of a car, viewed from the perspective of a passenger. The car appears to be in motion, as indicated by the blurred scenery outside the window, suggesting it's a moving shot. The interior is well-lit, with the backseat area featuring a headrest, a cup holder, and a storage compartment. There's a person visible in the driver's seat, but only their upper body is shown, and they seem to be driving. The car is passing by a building with a blue sign that reads "BICYCLE" and a stop sign. The road is lined with palm trees, and there are other vehicles on the street, including a truck and a car in the distance. The sky is clear, and the overall atmosphere suggests a sunny day. The video captures a moment of a typical car ride, with the focus on the interior of the vehicle and the passing scenery.
clip 1: The video shows a view from the back seat of a car, likely a taxi or rideshare vehicle, as indicated by the presence

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B-Instruct', use_fast=True)
model = AutoModelForCausalLM.from_pretrained('meta-llama/Meta-Llama-3-8B-Instruct', torch_dtype='auto', attn_implementation='sdpa')
model.to('cuda')
model.eval()



Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  7.20it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


After analyzing the 20 video clips, I have integrated the details to create a unified and coherent description of the full video. Here's the resulting narrative:

The video begins with a passenger in the backseat of a car, driving down a street lined with palm trees, passing by a building with a blue sign reading "BICYCLE" and a stop sign. The road is wide, with a clear lane for vehicles on the right side and a separate lane for pedestrians on the left. The sky is clear and blue, indicating good weather conditions.

As the car continues to drive, it passes through an urban or suburban area with buildings on both sides of the road. The camera captures the view of the road ahead, including a fire hydrant on the right side of the street. The driver is visible in the rearview mirror, and the car is moving at a moderate speed.

The video then cuts to a first-person perspective from inside a vehicle, likely a car or a golf cart, as it drives down a straight, wide road with a clear blue sky a

In [ ]:
prompt = "I have provided you with the descriptions of multiple video clips that are segments of one continuous video. Your task is to seamlessly integrate the details from each clip into a unified, coherent description of the full video. The goal is to capture the flow, transitions, and overall context while maintaining a natural progression. Here are the video clips:"

conversation = [
        {'role': 'user', 'content': prompt + '\n' + example},
    ]

input_ids = tokenizer.apply_chat_template(conversation, return_tensors='pt', add_generation_prompt=True).cuda()
output_ids = model.generate(input_ids, max_length=8192)[:,input_ids.size(1):]
text = tokenizer.decode(output_ids[0])
print(text)

In [31]:
prompt = 'I will provide you with a description of a video that includes both environmental details and a description of an action I am performing. Your task is to identify the key objects or features in the environment that I need to pay attention to in order to successfully complete the action. These could be items directly involved in the action or other noteworthy objects in the surroundings that may affect my task. Please focus on selecting elements that are crucial for ensuring the action is completed smoothly and safely.'

action_narration = [{
                "time": 0.0,
                "text": "You press the starter button of the rickshaw."
            },
            {
                "time": 2.81561,
                "text": "You look around."
            },
            {
                "time": 7.11184,
                "text": "You drive the rickshaw backwards."
            },
            {
                "time": 13.4554,
                "text": "You look around."
            },
            {
                "time": 21.88683,
                "text": "You drive the rickshaw down the road."
            },
            {
                "time": 39.58893,
                "text": "You look around."
            },
            {
                "time": 62.12729,
                "text": "You point to the road with your finger."
            },
            {
                "time": 103.24132,
                "text": "You stop on the road."
            },
            {
                "time": 112.48425,
                "text": "You drive the rickshaw down the road."
            },
            {
                "time": 203.69019,
                "text": "You look around."
            },
            {
                "time": 206.64051,
                "text": "You drive the rickshaw down the road."
            },
            {
                "time": 227.10612,
                "text": "You look around."
            }]
action_narration_text = '\n'.join([f"{step['time']:.2f}s: {step['text']}" for step in action_narration])
conversation = [
        {'role': 'user', 'content': prompt + '\n' + 'video description:' +  '\n' + text + '\n' + ' action desciption:' +  '\n' + action_narration_text},
    ]

input_ids = tokenizer.apply_chat_template(conversation, return_tensors='pt', add_generation_prompt=True).cuda()
output_ids = model.generate(input_ids, max_length=8192)[:,input_ids.size(1):]
text = tokenizer.decode(output_ids[0])
print(text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the video description and the action description, I've identified the key objects or features in the environment that you need to pay attention to in order to successfully complete the action:

1. **Road**: The road is a crucial element throughout the video and action. You need to be aware of the road's condition, width, and any obstacles or hazards, such as pedestrians, buildings, or other vehicles.
2. **Pedestrians**: As you drive the rickshaw, you should be aware of pedestrians on the road, especially in urban or suburban areas, to ensure a safe and smooth journey.
3. **Buildings**: The video features buildings on both sides of the road, which can affect your path and visibility. You should be mindful of these structures and adjust your driving accordingly.
4. **Stop signs**: You encounter a stop sign at 20.00s, which requires you to come to a complete stop before proceeding.
5. **Fire hydrant**: The fire hydrant on the right side of the street (around 5.00s) is an object y

In [ ]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
from decord import VideoReader, cpu    # pip install decord

model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True,
    attn_implementation='sdpa', torch_dtype=torch.bfloat16) # sdpa or flash_attention_2, no eager
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)

MAX_NUM_FRAMES=64 # if cuda OOM set a smaller number

def encode_video(video_path):
    def uniform_sample(l, n):
        gap = len(l) / n
        idxs = [int(i * gap + gap / 2) for i in range(n)]
        return [l[i] for i in idxs]

    vr = VideoReader(video_path, ctx=cpu(0))
    sample_fps = round(vr.get_avg_fps() / 1)  # FPS
    frame_idx = [i for i in range(0, len(vr), sample_fps)]
    if len(frame_idx) > MAX_NUM_FRAMES:
        frame_idx = uniform_sample(frame_idx, MAX_NUM_FRAMES)
    frames = vr.get_batch(frame_idx).asnumpy()
    frames = [Image.fromarray(v.astype('uint8')) for v in frames]
    print('num frames:', len(frames))
    return frames

video_path ="video_test.mp4"
frames = encode_video(video_path)
question = "Describe the video"
msgs = [
    {'role': 'user', 'content': frames + [question]}, 
]

# Set decode params for video
params={}
params["use_image_id"] = False
params["max_slice_nums"] = 2 # use 1 if cuda OOM and video resolution >  448*448

answer = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer,
    **params
)
print(answer)


In [2]:
# test.py
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True,
    attn_implementation='sdpa', torch_dtype=torch.bfloat16) # sdpa or flash_attention_2, no eager
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)

image = Image.open('/root/videollm-online/data/preprocess/output.png').convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': [image, question]}]

res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer
)
print(res)

## if you want to use streaming, please make sure sampling=True and stream=True
## the model.chat will return a generator
res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer,
    sampling=True,
    stream=True
)

generated_text = ""
for new_text in res:
    generated_text += new_text
    print(new_text, flush=True, end='')


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  5.85it/s]
A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6:
- tokenization_minicpmv_fast.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6:
- image_processing_minicpmv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6:
- processing_minicpmv.py
- image_processing_minicpmv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/root/miniconda3/envs/videollm/lib/python3.10/sit

The image shows the interior of a vehicle, likely taken from the perspective of someone sitting in the front seat. Through the windshield, we can see a street with buildings on either side and a car parked to the left. There's also a sign visible in the distance.
The image shows a view from inside a vehicle, likely an off-road or utility type of car. You can see the interior roof and part of the windshield frame. Outside, there's a street with a white pickup truck on the left side. There is also a pedestrian crossing sign visible in the distance. The perspective suggests that the photo was taken by someone sitting in the driver's seat looking forward.